In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()


from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()


In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load("dummy_folder/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]


In [7]:
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load("dummy_folder/sample_lda_libsvm_data.txt")

# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

The lower bound on the log likelihood of the entire corpus: -829.0944634832906
The upper bound on perplexity: 3.1888248595511177
The topics described by their top-weighted terms:


C:\Users\venka\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[6, 2, 5]  |[0.10747702477482395, 0.09676796961774264, 0.09409980637419452]|
|1    |[9, 10, 1] |[0.09675022749576347, 0.09643222070268911, 0.09582033048333911]|
|2    |[1, 2, 10] |[0.09912692286898944, 0.09642532810387178, 0.09610636937927883]|
|3    |[1, 4, 10] |[0.09936688778223718, 0.09789720274037217, 0.09674466310409335]|
|4    |[4, 7, 10] |[0.10666242844577777, 0.1017790146682606, 0.10162190997576949] |
|5    |[10, 8, 2] |[0.10795564517325613, 0.10107307704896823, 0.09396280094235228]|
|6    |[8, 6, 5]  |[0.10691737905965039, 0.10026590527556538, 0.0982965523601434] |
|7    |[10, 4, 1] |[0.10378305795309944, 0.10182226025652234, 0.09813180441771625]|
|8    |[3, 10, 6] |[0.21719272714675678, 0.21387471784256895, 0.137484943129

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("summy_folder/sample_movielens_ratings.txt")
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

AnalysisException: Path does not exist: file:/C:/Users/venka/summy_folder/sample_movielens_ratings.txt

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[100,101,102...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[128,129,130...|
|       0.0|  0.0|(692,[150,151,152...|
|       0.0|  0.0|(692,[152,153,154...|
+----------+-----+--------------------+
only showing top 5 rows

Test Error = 0.0526316 
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_67e95c7592b0, depth=2, numNodes=5, numClasses=2, numFeatures=692
